In [ ]:
from datetime import timezone
from pathlib import Path

In [ ]:
import numpy as np
import pandas as pd
from plotly.subplots import make_subplots
import plotly.graph_objects as go
from sklearn.model_selection import TimeSeriesSplit, validation_curve
from sklearn.svm import SVR

In [ ]:
from tfmmeteogalicia.dataset import load_wrf_hist_dataset
from tfmmeteogalicia.thredds_wrf import MeteoGaliciaNetCDFSubsetColumns, ThreddsWRFDomain, ThreddsWRFServerRun
from tfmforecasting.dataset import AdditionalHousingUnitFields, HousingUnitColumns
from tfmforecasting.preprocessing import (
    add_datetime_features, add_rbf, add_datetime_to_housing_unit_dataset, lag_consumption_feature, name_lagged_feature
)
from tfmforecasting.utils import find_cluster_files, get_housing_unit_name

In [ ]:
Cs = np.logspace(-8, 4, 11)
year = 2024
month = 5

In [ ]:
domain: ThreddsWRFDomain = "d02"
server_run: ThreddsWRFServerRun = "0000"
weather_data_dir = Path("../data/meteogalicia/thredds/wrf_hist").resolve()

In [ ]:
start_date = pd.Timestamp(year=year, month=month, day=1, tz=timezone.utc)
end_date = pd.Timestamp(year=year, month=month + 1, day=1, tz=timezone.utc) - pd.Timedelta(days=1)
dates = pd.date_range(start=start_date, end=end_date)
weather_by_days = [
    load_wrf_hist_dataset(weather_data_dir, domain, server_run, date)
    for date in dates
]
weather_by_days = [
    df[df[MeteoGaliciaNetCDFSubsetColumns.DATE.value] < date + pd.Timedelta(days=1, hours=1)].copy().reset_index(drop=True)
    for df, date in zip(weather_by_days, dates)
]
weather_data = pd.concat(weather_by_days)
weather_data = weather_data.rename(columns={MeteoGaliciaNetCDFSubsetColumns.DATE.value: AdditionalHousingUnitFields.Datetime.value})
weather_data.head(n=4)

In [ ]:
cluster_id = 0
n_lags = 48
lagged_features = sorted([
    name_lagged_feature(HousingUnitColumns.Consumption.value, lag) for lag in range(24, 32)
], reverse=True)
consumption_data_dir = Path('../../analisis_consumos/data/viviendas/por_mes_con_cluster/cluster_4').resolve()

In [ ]:
cluster_files = find_cluster_files(consumption_data_dir, cluster_id, year=year, month=month)
cluster_files.sort()
housing_units = set([get_housing_unit_name(file) for file in cluster_files])
n_housing_units = len(housing_units)
housing_units

In [ ]:
csv_delimiter = ';'
rbf_target_column = AdditionalHousingUnitFields.Hour.value
rbf_period = 7
rbf_input_range = (0, 24)
rbf_columns = [f"{rbf_target_column}_rbf_{i}" for i in range(rbf_period)]

In [ ]:
data_frames = []
for cluster_file in cluster_files:
    df = pd.read_csv(cluster_file, delimiter=csv_delimiter)
    df = add_datetime_to_housing_unit_dataset(df)
    df = add_datetime_features(df)
    df = lag_consumption_feature(df, n_lags=n_lags)
    df[AdditionalHousingUnitFields.HousingUnit.value] = get_housing_unit_name(cluster_file)
    rbf_df = add_rbf(
        df.set_index(AdditionalHousingUnitFields.Datetime.value),
        column=rbf_target_column, period=rbf_period, input_range=rbf_input_range
    )
    df = df.merge(rbf_df, left_on=AdditionalHousingUnitFields.Datetime.value, right_on=rbf_df.index)
    data_frames.append(df)

In [ ]:
cluster_data = pd.concat(data_frames)
cluster_data = cluster_data.sort_values(
    by=[AdditionalHousingUnitFields.Datetime.value, AdditionalHousingUnitFields.HousingUnit.value]
).reset_index(drop=True)
consumption_columns = lagged_features + [HousingUnitColumns.Consumption.value]
cluster_data = cluster_data[
    [
        AdditionalHousingUnitFields.Datetime.value,
        AdditionalHousingUnitFields.HousingUnit.value
    ] + rbf_columns + consumption_columns
].copy().dropna().reset_index(drop=True)
cluster_data[cluster_data[AdditionalHousingUnitFields.HousingUnit.value] == 'ATF'].head(n=24)

In [ ]:
rbf_sample_df = data_frames[0]
rbf_sample_df

In [ ]:
fig = make_subplots(rows=len(rbf_columns), cols=1, shared_xaxes=True, vertical_spacing=0.05)
may_week = rbf_sample_df[
    (rbf_sample_df[AdditionalHousingUnitFields.Datetime.value] >= '2024-05-05') &
    (rbf_sample_df[AdditionalHousingUnitFields.Datetime.value] < '2024-05-11')
]
for row, column in zip(range(len(rbf_columns)), rbf_columns):
    fig.add_trace(
        go.Scatter(x=may_week.index, y=may_week[column], name=f"{column}"),
        row=row + 1, col=1
    )
fig.update_layout(height=600, title_text="Stacked Subplots with Shared X-Axes")
fig.show()

In [ ]:
merged_df = pd.merge(
    cluster_data,
    weather_data[[AdditionalHousingUnitFields.Datetime.value, MeteoGaliciaNetCDFSubsetColumns.TEMP.value]],
    on=AdditionalHousingUnitFields.Datetime.value
).sort_values(by=[AdditionalHousingUnitFields.Datetime.value, AdditionalHousingUnitFields.HousingUnit.value]).reset_index(drop=True)
merged_df.head(n=4)

In [ ]:
features = lagged_features + rbf_columns + [MeteoGaliciaNetCDFSubsetColumns.TEMP.value]
target = HousingUnitColumns.Consumption.value
features, target

In [ ]:
max_temp = 34.2 + 273.15
min_temp = 4 + 273.15
min_temp, max_temp

In [ ]:
max_consumption = merged_df[HousingUnitColumns.Consumption].max()
min_consumption = merged_df[HousingUnitColumns.Consumption].min()
min_consumption, max_consumption

In [ ]:
window_size = 21  # days
target_day = pd.Timestamp(year=year, month=month, day=27, hour=0, minute=0, second=0, microsecond=0, tz=timezone.utc)
window_start = target_day - pd.Timedelta(days=window_size)
sliced_df = merged_df[
    (
            merged_df[AdditionalHousingUnitFields.Datetime] >= window_start + pd.Timedelta(days=-1)
    ) & (
            merged_df[AdditionalHousingUnitFields.Datetime] < target_day + pd.Timedelta(days=1)
    )
    ].copy().reset_index(drop=True)
for lagged_feature in lagged_features:
    if lagged_feature not in features:
        continue
    sliced_df[lagged_feature] = (sliced_df[lagged_feature] - min_consumption) / (max_consumption - min_consumption)
sliced_df[MeteoGaliciaNetCDFSubsetColumns.TEMP] = (sliced_df[MeteoGaliciaNetCDFSubsetColumns.TEMP] - min_temp) / (
        max_temp - min_temp)
sliced_df.shape

In [ ]:
X = sliced_df[features].to_numpy()
y = sliced_df[HousingUnitColumns.Consumption].to_numpy()
sliced_df[features]

In [ ]:
n_samples = n_housing_units * window_size * 24
target_size = n_housing_units * 24
n_splits = 2
tscv = TimeSeriesSplit(max_train_size=n_samples, n_splits=n_splits, test_size=target_size)
for i, (train_index, test_index) in enumerate(tscv.split(X)):
    print(f"Fold {i}:")
    print(f"  Train: index={train_index}")
    print(f"  Test:  index={test_index}")

In [ ]:
Cs = np.logspace(-8, 4, 16) * np.sqrt(1 / n_samples)
Cs

In [ ]:
model = SVR(kernel='rbf', gamma='scale', epsilon=0.01)
train_scores, test_scores = validation_curve(
    model,
    X,
    y,
    param_name="C",
    param_range=Cs,
    cv=tscv,
    n_jobs=8,
    scoring='neg_mean_absolute_percentage_error',
)

In [ ]:
results_df = pd.DataFrame({
    "C": Cs,
    "test_scores": test_scores[:, 1] * -100,
    "train_scores": train_scores[:, 1] * -100,
})
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=results_df['C'],
        y=results_df['test_scores'],
        mode='lines+markers',
        name='Test scores',
    )
)
fig.add_trace(
    go.Scatter(
        x=results_df['C'],
        y=results_df['train_scores'],
        mode='lines+markers',
        name='Train scores',
    )
)
fig.update_xaxes(title_text="C", type="log")
fig.update_yaxes(title_text="MAPE")
fig.show()